In [ ]:
# 修复文件  保证书名跟书评是同一行

# 创建修复后内容的保存文件
fixed = open("douban_comments_fixed.txt" , "w" , encoding = "utf-8")

# 遍历修复前内容文件
lines = [line for line in open("doubanbook_top250_comments.txt","r" , encoding = "utf-8")]

# print(len(lines))

for i,line in enumerate(lines):
    # 保存标题列
    if i == 0 :
        fixed.write(line)
        prev_line = ' ' #  上一行的书名置为空
        continue
    # 根据制表符/t 进行拆分  提取书名和评论文本
    terms = line.split("\t")

    # 判断当前行的书名 是否等于 上一行的书名
    if terms[0] == prev_line.split("\t")[0]:
        if len(prev_line.split("\t")) == 6:  # 新书评论
            # 保存上一行的记录，直接写入 fixed
            fixed.write(prev_line + '\n')
            prev_line = line.strip() # 保存当前行
        else:
            prev_line = ""
    else:
        if len(terms) == 6:
            prev_line = line.strip() # 保存当前行
        else:
            prev_line += line.strip() # 合并当前行与上一行
fixed.close()

In [74]:
#  2、使用自定义的文档文本，通过fasttext训练word2vec训练词向量模型，并计算词汇间的相关度。

import jieba
import fasttext
# 文档分词预处理
# with open("shz.txt" , "r",encoding = "utf-8" ) as f :
#     lines = f.read()
#
# with open("shz_sp.txt" ,"w", encoding = "utf-8") as f :
#     f.write(" " . join(jieba.cut(lines)))

model = fasttext.train_unsupervised("shz_sp.txt",model = "skipgram")

print(len(model.words))

# 获取词向量的最近邻
print(model.get_nearest_neighbors("武松"))

print("-------------")
# 词汇间的类比
print(model.get_analogies('宋江' , '鲁智深'  , '林冲'))


26
[(0.8185369372367859, '，'), (0.8025778532028198, '。'), (0.7752787470817566, '、'), (0.7728549242019653, '梁山'), (0.7717704176902771, '好汉'), (0.7697096467018127, '在'), (0.7623936533927917, '宋江'), (0.7565146684646606, '鲁智深'), (0.7500678896903992, '的'), (0.6883617043495178, '人')]
-------------
[(0.4294202923774719, '在'), (0.40036338567733765, '。'), (0.3922351896762848, '的'), (0.38388118147850037, '也'), (0.3833412826061249, '武松'), (0.38171130418777466, '，'), (0.3669948875904083, '了'), (0.3530404567718506, '梁山'), (0.3516506552696228, '、'), (0.3049568235874176, '人')]


In [65]:
# 计算 TF-IDF并通过预选相似度给出的推荐列表
import csv
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer   # 特征提取
from sklearn.metrics.pairwise import cosine_similarity  # 余弦相似度
import numpy as np

def load_data(filename):
    # 图书评论信息集合{书名："评论1+评论2+。。。。"}
    book_comments = {}
    with open(filename , 'r' , encoding = 'utf-8') as f :
        # 识别格式文本中的标题列 csv.DictReader , delimiter   以制表符作为文件的分隔符
        reader = csv.DictReader(f , delimiter = "\t" )

        # 提取书名与评论信息 并且将评论信息进行分词
        for item in reader:
            book = item['book']
            comment = item['body']
            comment_words = jieba.lcut(comment)

            if book == ' ' : continue # 跳过空书名

            # 将图书评论收集，一个书名对应该书名的全部评论
            book_comments[book] = book_comments.get(book , [])
            # 加上后续评论
            book_comments[book].extend(comment_words)
    return book_comments

if __name__ == '__main__':
    # 加载停用词列表
    stop_words = [line.strip() for line in open("stopwords.txt" , 'r' , encoding = "utf-8")]

    # 加载图书评论信息
    book_comments = load_data("douban_comments_fixed.txt")
    # print(len(book_comments))

    # 提取书名
    book_names = []
    book_comms = []
    for book , comments in book_comments.items():
        book_names.append(book)
        book_comms.append(comments)
    # 构建TF-IDF特征矩阵
    vectorizer = TfidfVectorizer(stop_words = stop_words)
    tfidf_matrix = vectorizer.fit_transform([" " . join(comms) for comms in book_comms])

    # 计算图书之间的余弦相似度
    similarity_matrix = cosine_similarity(tfidf_matrix)


    # 输入要推荐的图书名称
    book_list = list(book_comments.keys())
    print(book_list)

    book_name = input("请输入图书名称：")
    book_idx = book_names.index(book_name) # 获取图书索引

    # 获取与输入图书最相似的图书 输入图书除外（第一本）
    # argsort  返回升序索引值，所以similarity_matrix 前面加一个符号，用于降序
    recommend_book_index = np.argsort(-similarity_matrix[book_idx])[1:11]
    print(f"推荐的十种书：")
    # 输出推荐的图书
    for idx in recommend_book_index:
        print(f"《{book_names[idx]}》 \t 相似度：{similarity_matrix[book_idx][idx]:.4f}")
    print()


KeyboardInterrupt: 

In [83]:
# 作业1
import csv
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from bm25_code import bm25

# 对评论进行分词并且收集所有书名的评论汇总到一起
def load_data(filename):
    book_comments = {}

    with open(filename , 'r' , encoding = "utf-8") as f:
        # 识别格式
        reader = csv.DictReader(f , delimiter = "\t",)
        for item in reader:
            book = item['book']
            comments = item['body']
            # 利用jieba的lcut 进行分词处理
            comments_words = jieba.lcut(comments)
            if book == '' : continue  # 跳过空书名
            # 评论集合
            book_comments[book] = book_comments.get(book,[])
            book_comments[book].extend(comments_words)
    return book_comments

# 判断用模型训练tfidf 还是 bm25
def comments_vectors_similarity(book_comms , method = 'tfidf'):
    global matrix
    if method == 'tfidf':
        # 构建TF-IDF特征矩阵
        vectorizer = TfidfVectorizer()
        matrix = vectorizer.fit_transform([''.join(comms) for comms in book_comms])
    if method == 'dm25':
        matrix = bm25(book_comms)
    # 计算图书余弦相似度
    similarity_matrix = cosine_similarity(matrix)
    return similarity_matrix

if __name__ == '__main__':

    # 加载停用词列表
    stop_words = [line.strip() for line in open("stopwords.txt" , 'r' , encoding = "utf-8")]


    # 加载图书评论信息
    book_comments = load_data('douban_comments_fixed.txt')

    # 提取书名和评论文本
    book_names = []
    book_comms = []
    for book,comments in book_comments.items():
        book_names.append(book)
        book_comms.append(comments)

    # TF-IDF 算法得到的相似度矩阵
    tfidf_matrix = comments_vectors_similarity(book_comms , method = 'tfidf')

    bm25_matrix = comments_vectors_similarity(book_comms , method = 'bm25')


    # 输入要推荐的图书名称
    book_list = list(book_comments.keys())
    print(book_list)
    book_name = input("请输入图书名称：")
    book_idx = book_names.index(book_name)

    # 获取与输入图书最相似的图书
    print(f"TF-IDF算法推荐的图书：\n")
    recommend_book_index = np.argsort(-tfidf_matrix[book_idx])[1:11]
    # 输出推荐的图书
    for idx in recommend_book_index:
        print(f"《{book_names[idx]}》 \t 相似度：{tfidf_matrix[book_idx][idx]:.4f}")
    print()

    print(f"BM25算法推荐的图书：\n")
    recommend_book_index = np.argsort(-bm25_matrix[book_idx])[1:11]
    # 输出推荐的图书
    for idx in recommend_book_index:
        print(f"《{book_names[idx]}》 \t 相似度：{bm25_matrix[book_idx][idx]:.4f}")



['天才在左 疯子在右', '1Q84 BOOK 1', '悲伤逆流成河', '恶意', 'Harry Potter and the Deathly Hallows', '长安乱', '苏菲的世界', '许三观卖血记', '1995-2005夏至未至', '盗墓笔记', '霍乱时期的爱情', '三生三世 十里桃花', '基督山伯爵', '小时代1.0折纸时代', '洛丽塔', '1Q84 BOOK 2', '第一次的亲密接触', '神雕侠侣', '一座城池', '茶花女', '当我谈跑步时我谈些什么', '明朝那些事儿（贰）', '人类简史', '一個人住第5年', '明朝那些事儿（肆）', '寻路中国', '我们台湾这些年', '1Q84 BOOK 3', '摆渡人', '明朝那些事儿（伍）', '骆驼祥子', '盗墓笔记3', '麦琪的礼物', '格林童话全集', '水仙已乘鲤鱼去', '历史深处的忧虑', '金锁记', '草样年华', '刀锋', '飞鸟集', '七夜雪', '最初的爱情 最后的仪式', '拆掉思维里的墙', '明朝那些事儿（陆）', '追风筝的人', '小王子', '围城', '解忧杂货店', '活着', '白夜行', '挪威的森林', '嫌疑人X的献身', '三体', '不能承受的生命之轻', '红楼梦', '梦里花落知多少', '达·芬奇密码', '看见', '百年孤独', '1988：我想和这个世界谈谈', '何以笙箫默', '平凡的世界（全三部）', '简爱', '哈利·波特与魔法石', '三体Ⅱ', '飘', '送你一颗子弹', '三体Ⅲ', '傲慢与偏见', '倾城之恋', '三重门', '杜拉拉升职记', '明朝那些事儿（壹）', '哈利·波特与阿兹卡班的囚徒', '目送', '情人', '哈利·波特与密室', '万历十五年', '我们仨', '幻城', '致我们终将逝去的青春', '狼图腾', '微微一笑很倾城', '莲花', '哈利·波特与火焰杯', '边城', '月亮和六便士', '向左走·向右走', '穆斯林的葬礼', '从你的全世界路过', '天龙八部', '放学后', '哈利·波特与混血王子', '一个人的好天气', '哈利·波特与凤凰社', '喜宝', '海边的卡夫卡', '文化苦旅', '窗边的小豆豆', '三国

In [66]:
# 3. 使用课堂示例cooking.stackexchange.txt，使用fasttext训练文本分类模型。

import fasttext

model = fasttext.train_supervised('cooking.stackexchange.txt',epoch = 30 , dim = 100)

# print(model.words)

# 返回预测概率最高的标签
print(model.predict('What kind of tea do you boil for 45minutes?'))
print(model.predict('Good breads for evening cooking?'))

# 保存模型
model.save_model('model_file.bin' )

# 获取模型
model = fasttext.load_model('model_file.bin')

(('__label__tea',), array([0.38441685]))
(('__label__bread',), array([0.85953915]))
